# Distribution of Hospital Beds per Departments
Scripts for creating the visualisation of the hospital beds in France grouped by type and department.
@author: GiuliaMuzio

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from IPython import embed
import pandas as pd

In [2]:
def save_file(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)


def load_file(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
def obtain_data(data):
    KEYS = list(data.keys())
    # removing departments oversea
    keys = np.array(KEYS[:96]).astype(object)
    idx1 = np.where(keys == "2A")[0]
    idx2 = np.where(keys == "2B")[0]
    keys = np.delete(np.array(keys), [idx1, idx2])
    name, capital, icu_beds, acute_beds = [], [], [], []
    for key in keys:
        name.append(data[key]["name"])
        capital.append(data[key]["capital"])
        icu_beds.append(data[key]["beds"]["icu"])
        acute_beds.append(data[key]["beds"]["acute"])

    df = pd.DataFrame(
        data=np.transpose([name, capital, icu_beds, acute_beds]),
        columns=["department", "capital", "icu beds", "acute beds"]
    )
    df['icu beds'] = df['icu beds'].astype(int)
    df['acute beds'] = df['acute beds'].astype(int)

    # dataframe for visualising both
    beds = np.concatenate((icu_beds, acute_beds))
    label = np.concatenate(
        (np.full(len(icu_beds), "ICU"), np.full(len(icu_beds), "acute"))
    )
    dep = np.concatenate((name, name))
    df_ = pd.DataFrame(
        data=np.transpose([beds, label, dep]), columns=["Beds", "Bed kind", "dep"]
    )
    df_['Beds'] = df_['Beds'].astype(int)
    
    # Region level plots
    region = region_f()

    # check if all the name are spelled in the same way + manual correction
    dep_name = np.sort(list(region.keys()))
    non_equal = np.where(np.sort(name) != dep_name)[0]
    for non in non_equal:
        print(np.sort(name)[non])
        print(dep_name[non])
        print("")

    reg = []
    for elem in name:
        reg.append(region[elem])

    df_["region"] = np.concatenate((reg, reg))
    reg_unique = np.unique(reg)
    return df, df_, name, capital, icu_beds, acute_beds, reg_unique

In [4]:
def region_f():
    region_dict = {
        "Bas-Rhin": "Alsace",
        "Haut-Rhin": "Alsace",
        "Dordogne": "Aquitaine",
        "Gironde": "Aquitaine",
        "Landes": "Aquitaine",
        "Lot-et-Garonne": "Aquitaine",
        "Pyrénées Atlantiques": "Aquitaine",
        "Allier": "Auvergne",
        "Cantal": "Auvergne",
        "Haute Loire": "Auvergne",
        "Puy-de-Dôme": "Auvergne",
        "Calvados": "Basse-Normandie",
        "Manche": "Basse-Normandie",
        "Orne": "Basse-Normandie",
        "Côte-d'Or": "Bourgogne",
        "Nièvre": "Bourgogne",
        "Saône-et-Loire": "Bourgogne",
        "Yonne": "Bourgogne",
        "Côtes d'Armor": "Bretagne",
        "Finistère": "Bretagne",
        "Ille-et-Vilaine": "Bretagne",
        "Morbihan": "Bretagne",
        "Cher": "Centre",
        "Eure-et-Loir": "Centre",
        "Indre": "Centre",
        "Indre-et-Loire": "Centre",
        "Loir-et-Cher": "Centre",
        "Loiret": "Centre",
        "Ardennes": "Champagne-Ardenne",
        "Aube": "Champagne-Ardenne",
        "Marne": "Champagne-Ardenne",
        "Haute Marne": "Champagne-Ardenne",
        "Doubs": "Franche-Comté",
        "Jura": "Franche-Comté",
        "Haute Saône": "Franche-Comté",
        "Territoire de Belfort": "Franche-Comté",
        "Eure": "Haute-Normandie",
        "Seine Maritime": "Haute-Normandie",
        "Paris": "Ile-de-France",
        "Seine-et-Marne": "Ile-de-France",
        "Yvelines": "Ile-de-France",
        "Essonne": "Ile-de-France",
        "Hauts-de-Seine": "Ile-de-France",
        "Seine-St-Denis": "Ile-de-France",
        "Val-de-Marne": "Ile-de-France",
        "Val-D'Oise": "Ile-de-France",
        "Aude": "Languedoc-Roussillon",
        "Gard": "Languedoc-Roussillon",
        "Hérault": "Languedoc-Roussillon",
        "Lozère": "Languedoc-Roussillon",
        "Pyrénées Orientales": "Languedoc-Roussillon",
        "Corrèze": "Limousin",
        "Creuse": "Limousin",
        "Haute Vienne": "Limousin",
        "Meurthe-et-Moselle": "Lorraine",
        "Meuse": "Lorraine",
        "Moselle": "Lorraine",
        "Vosges": "Lorraine",
        "Ariège": "Midi-Pyrénées",
        "Aveyron": "Midi-Pyrénées",
        "Haute Garonne": "Midi-Pyrénées",
        "Gers": "Midi-Pyrénées",
        "Lot": "Midi-Pyrénées",
        "Hautes Pyrénées": "Midi-Pyrénées",
        "Tarn": "Midi-Pyrénées",
        "Tarn-et-Garonne": "Midi-Pyrénées",
        "Nord": "Nord-Pas-de-Calais",
        "Pas-de-Calais": "Nord-Pas-de-Calais",
        "Loire Atlantique": "Pays of Loire",
        "Maine-et-Loire": "Pays of Loire",
        "Mayenne": "Pays of Loire",
        "Sarthe": "Pays of Loire",
        "Vendée": "Pays of Loire",
        "Aisne": "Picardie",
        "Oise": "Picardie",
        "Somme": "Picardie",
        "Charente": "Poitou-Charentes",
        "Charente-Maritime": "Poitou-Charentes",
        "Deux-Sèvres": "Poitou-Charentes",
        "Vienne": "Poitou-Charentes",
        "Alpes-de-Haute-Provence": "Provence-Alpes-Côte-d'Azur",
        "Hautes-Alpes": "Provence-Alpes-Côte-d'Azur",
        "Alpes Maritimes": "Provence-Alpes-Côte-d'Azur",
        "Bouches-du-Rhône": "Provence-Alpes-Côte-d'Azur",
        "Var": "Provence-Alpes-Côte-d'Azur",
        "Vaucluse": "Provence-Alpes-Côte-d'Azur",
        "Ain": "Rhône-Alpes",
        "Ardèche": "Rhône-Alpes",
        "Drôme": "Rhône-Alpes",
        "Isère": "Rhône-Alpes",
        "Loire": "Rhône-Alpes",
        "Rhône": "Rhône-Alpes",
        "Savoie": "Rhône-Alpes",
        "Haute Savoie": "Rhône-Alpes",
    }

    return region_dict

In [5]:
data = load_file('./data/processed_data/departments.pkl')
df, df_, name, capital, icu_beds, acute_beds, reg_unique = obtain_data(data)

# ICU beds
fig = plt.figure(figsize=(12,10))
ax = sns.barplot(x = 'department', y = 'icu beds', data = df)
plt.xticks(rotation=90, fontsize = 8)
plt.xlabel('France departments')
plt.ylabel('ICU beds number')
plt.title('ICU beds per department - France')
plt.tight_layout()
plt.savefig('./visualisations/beds/icu_beds.pdf')
plt.close(fig)

# acute beds
fig = plt.figure(figsize=(12,10))
ax = sns.barplot(x = 'department', y = 'acute beds', data = df)
plt.xticks(rotation=90, fontsize = 8)
plt.xlabel('France departments')
plt.ylabel('Acute beds number')
plt.title('Acute beds per department - France')
plt.tight_layout()
plt.savefig('./visualisations/beds/acute_beds.pdf')
plt.close(fig)

# both acute and ICU beds
fig = plt.figure(figsize=(12,10))
ax = sns.barplot(x = 'dep', y = 'Beds', hue = 'Bed kind', data = df_)
plt.xticks(rotation=90, fontsize = 8)
plt.xlabel('France departments')
plt.ylabel('Beds number')
plt.title('Beds per department - France')
plt.tight_layout()
plt.savefig('./visualisations/beds/beds.pdf')
plt.close(fig)

for r in reg_unique:
    fig = plt.figure(figsize=(12,10))
    ax = sns.barplot(x = 'dep', y = 'Beds', hue = 'Bed kind', data = df_[df_['region']==r])
    plt.xticks(rotation=90, fontsize = 8)
    plt.xlabel(r + ' departments')
    plt.ylabel('Beds number')
    plt.title('Beds per department - ' + r)
    plt.tight_layout()
    plt.savefig('./visualisations/beds/beds_' + r +'.pdf')
    plt.close(fig)

# Daily Patient-flow animations
@author: percyfan

In [6]:
df = pd.read_csv('./data/France_Hospital_data/donnees-hospitalieres-nouveaux-covid19-2020-04-23-19h00.csv', header=0,sep=';')
df = df.groupby(['jour',  'dep']).sum()
time_index = list(np.unique(df.index.get_level_values(0)))

In [7]:
with open('./data/processed_data/departments.pkl', 'rb') as handle:
    France_atts = pickle.load(handle)
loci = pd.DataFrame.from_dict(France_atts)
loci.columns = np.unique(df.index.get_level_values(1))

In [8]:
import branca.colormap
from collections import defaultdict
import folium
import folium.plugins as plugins

for cat in df.columns:
    data_hosp = []
    for _, df_temp in df.groupby(level=0):
        data_temp = []
        for weight, (key, value) in zip(df_temp[cat], loci.loc['capital_coords'].items()):
            data_temp.append([value['lat'], value['lng'], weight/(df[cat].max())])
        data_hosp.append(data_temp)
    
    m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

    hm = plugins.HeatMapWithTime(
        data_hosp,
        radius=45,
        index=time_index,
        auto_play=True,
        max_opacity=0.3
    )
    hm.add_to(m)
    
    file_path = r"./visualisations/daily_flow/{}.html".format(str(cat))
    hm.save(file_path)